In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
from utils.preprocessing import get_gray_images
import sys

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

2022-11-23 15:03:31.416555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 15:03:32.034260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 15:03:32.034283: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 15:03:32.108188: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 15:03:33.297415: W tensorflow/stream_executor/platform/de

In [2]:
testing_dir = '../data/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data'
training_dir_base = f'../data/C-NMC_Leukemia/training_data/'
fold_0_all_path = 'fold_0/all'
fold_0_hem_path = 'fold_0/hem'

In [3]:
### The code for this lives in notebooks.utils.preprocessing
### This is separated from the code below to optomize memory usage
gray_img_train_cancer = get_gray_images(training_dir_base+fold_0_all_path)

2022-11-23 15:03:48.608670: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 15:03:48.609243: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 15:03:48.609271: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (karl): /proc/driver/nvidia/version does not exist
2022-11-23 15:03:48.612770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
### Get images for healthy cells, and labels for both classes
gray_img_train_healthy = get_gray_images(training_dir_base+fold_0_hem_path)
train_health_labels = [0 for _ in range(len(gray_img_train_healthy))]
train_cancer_labels = [1 for _ in range(len(gray_img_train_cancer))]
#gray_img_test = get_gray_images(testing_dir)

In [5]:
### For random forests, X is our combined image set and y is our combined label set.
X =  np.array(gray_img_train_cancer + gray_img_train_healthy)
y = np.array(train_cancer_labels + train_health_labels)

## Random Forests

In [11]:
### train-test-split our data, and initialize the forest.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=100)

forest = RandomForestClassifier(n_estimators=100, random_state=100)

In [7]:
### The random forests algorithm expects 2 dimensional data at the most.
### We can accomplish this on our 4-dimensional data by multiplying the last 3 dimensions together to flatten the data
X_shape = X_train.shape
X_tshape = X_test.shape
X_train = X_train.reshape(X_shape[0], X_shape[1] * X_shape[2] * X_shape[3])
X_test = X_test.reshape(X_tshape[0], X_tshape[1] * X_tshape[2] * X_tshape[3])

In [8]:
### Fit the random forest, try it on the test data, and assess the model's accuracy
forest.fit(X_train,y_train)
predictions = forest.predict(X_test)

In [10]:
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

Accuracy: 0.8593012275731823
